# 회귀문제

## 1. 패키지 불러오기

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
import matplotlib.pyplot as plt

## 2. 데이터 불러오기

In [2]:
boston = load_boston()

- 보스턴 집값 데이터의 입력층의 노드 개수는 13개

In [5]:
boston['data'].shape

(506, 13)

- 보스턴 집값 데이터의 행은 506

In [4]:
print(len(boston['target']))

506


- 4장에서 배운 내용을 토대로 직접 간단한 신경망을 구현해보기

- 계획: 입력층 --> 은닉층 --> 은닉층 --> 출력층 ==> 4층짜리 신경망 만들기

- 입력층 노드: 13개 / 출력층 노드: 1개 (회귀문제니까)
- 첫번째 은닉층 노드: 64개 / 두번째 은닉층 노드: 32개 
- 입력층 -> 은닉층 -> 은닉층 -> 출력층 => 활성화함수: ReLU / 출력층에서 활성화함수: 항등함수

- 필요한 것들
- 활성화함수: ReLU, 항등항수
- 가중치1, 가중치2, 가중치3
- 편향1, 편향2, 편향3

## 활성화 함수들

- ReLU함수

In [6]:
def ReLU(x):
    return np.where(x<=0, 0, x)

- 항등함수

In [7]:
def identify(x):
    return x

- leaky ReLU

In [8]:
def leaky(x):
    return np.where(x <= 0, 0.0001*x, x)

- 소프트맥스 만들기

In [9]:
def softmax(x):
    return np.exp(-x)/sum(np.exp(-x))

- 시그모이드

In [10]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

- tanh 함수

In [11]:
def tanh(x):
    return (1-np.exp(-x))/(1+np.exp(-x))

### 가중치 만들기

In [39]:
def make_weight(input_node, first_hidden, second_hidden, output_node):
    # 첫번째 가중치
    W1 = np.zeros((input_node, first_hidden))
    
    for i in range(W1.shape[0]):
        for j in range(W1.shape[1]):
            W1[i][j] = np.random.randn()
            
    # 두번째 가중치
    W2 = np.zeros((first_hidden, second_hidden))
    
    for i in range(W2.shape[0]):
        for j in range(W2.shape[1]):
            W2[i][j] = np.random.randn()

    # 세번째 가중치
    W3 = np.zeros((second_hidden, output_node))
    
    for i in range(W3.shape[0]):
        for j in range(W3.shape[1]):
            W3[i][j] = np.random.randn()
    
    return W1, W2, W3

In [40]:
W1, W2, W3 = make_weight(13, 64, 32, 1)

### 편향 만들기

In [41]:
def make_bias(first_hidden, second_hidden, output_node):
    # 첫번째 편향
    b1 = np.zeros(first_hidden)
    
    for i in range(b1.shape[0]):
        b1[i] = np.random.randn()
        
    # 두번째 편향
    b2 = np.zeros(second_hidden)
    
    for i in range(b2.shape[0]):
        b2[i] = np.random.randn()
    
    # 두번째 편향
    b3 = np.zeros(output_node)
    
    for i in range(b3.shape[0]):
        b3[i] = np.random.randn()
    
    return b1, b2, b3

In [42]:
b1, b2, b3 = make_bias(64, 32, 1)

### 신경망 쌓아보기

In [43]:
y_result = []

In [44]:
for i in range(boston['data'].shape[0]):
    
    # 입력층 노드
    start = boston['data'][i]
    
    # 입력층 -> 은닉층
    u1 = np.dot(start, W1) + b1
    f1 = ReLU(u1)
    
    # 은닉층 -> 은닉층
    u2 = np.dot(f1, W2) + b2
    f2 = ReLU(u2)
    
    # 은닉층 -> 출력층
    u3 = np.dot(f2, W3) + b3
    f3 = ReLU(u3)
    
    # 출력층 -> 항등함수
    result = identify(f3)
    
    y_result.append(result)

In [45]:
target = boston['target']

In [46]:
target = pd.DataFrame(target)
result = pd.DataFrame(y_result)

In [47]:
pd.concat([target, result], axis=1)

,0,0
0,24.0,11656.310202
1,21.6,10213.033852
2,34.7,10427.329216
3,33.4,10128.782946
4,36.2,10054.150284
...,...,...
501,22.4,10978.371686
502,20.6,10975.108517
503,23.9,10847.326806
504,22.0,10813.371409


- 아무런 학습이나 최적화도 거치지 않고 신경망을 한 번 거쳐서 나온 결과는 이처럼 처참하다
- 기울기 소실문제 발생 --> relu 대신 leaky relu

In [48]:
y_result = []

In [49]:
for i in range(boston['data'].shape[0]):
    
    # 입력층 노드
    start = boston['data'][i]
    
    # 입력층 -> 은닉층
    u1 = np.dot(start, W1) + b1
    f1 = leaky(u1)
    
    # 은닉층 -> 은닉층
    u2 = np.dot(f1, W2) + b2
    f2 = leaky(u2)
    
    # 은닉층 -> 출력층
    u3 = np.dot(f2, W3) + b3
    f3 = leaky(u3)
    
    # 출력층 -> 항등함수
    result = identify(f3)
    
    y_result.append(result)

In [50]:
result = pd.DataFrame(y_result)

In [51]:
pd.concat([target, result], axis=1)

,0,0
0,24.0,11653.788798
1,21.6,10210.742853
2,34.7,10424.960057
3,33.4,10126.428987
4,36.2,10051.817964
...,...,...
501,22.4,10976.043153
502,20.6,10972.790005
503,23.9,10845.060912
504,22.0,10811.113191


- 역시 처참

# 분류문제

## 1. 패키지 불러오기

In [55]:
%matplotlib inline
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris

In [56]:
iris = load_iris()

- 행은 150개

In [57]:
iris['target'].shape

(150,)

- 출력층은 0,1,2 다중분류

In [58]:
iris['target']

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

- 입력층의 노드 개수 4개

In [59]:
iris['data'].shape

(150, 4)

- 계획: 입력층 --> 은닉층 --> 은닉층 --> 출력층 ==> 4층짜리 신경망 만들기

- 입력층 노드: 4개 / 출력층 노드: 3개
- 첫번째 은닉층 노드: 64개 / 두번째 은닉층 노드: 32개 
- 입력층 -> 은닉층 -> 은닉층 -> 출력층 => 활성화함수: ReLU / 출력층에서 활성화함수: 소프트맥스

- 필요한 것들
- 활성화함수: ReLU, 소프트맥스
- 가중치1, 가중치2, 가중치3
- 편향1, 편향2, 편향3

### 가중치 설정

In [60]:
W1, W2, W3 = make_weight(4, 64, 32, 3)

### 편향 설정 

In [61]:
b1, b2, b3 = make_bias(64, 32, 3)

### 신경망 쌓아보기

In [65]:
y_result = []

In [66]:
for i in range(iris['data'].shape[0]):
    
    # 입력층 노드
    start = iris['data'][i]
    
    # 입력층 -> 은닉층
    u1 = np.dot(start, W1) + b1
    f1 = ReLU(u1)
    
    # 은닉층 -> 은닉층
    u2 = np.dot(f1, W2) + b2
    f2 = ReLU(u2)
    
    # 은닉층 -> 출력층
    u3 = np.dot(f2, W3) + b3
    f3 = ReLU(u3)
    
    # 출력층 -> 항등함수
    result = softmax(f3)
    result = result.argmax()    
    
    y_result.append(result)

In [67]:
target = iris['target']

In [68]:
target = pd.DataFrame(target)
result = pd.DataFrame(y_result)

In [69]:
df = pd.concat([target, result], axis=1)

In [70]:
df.columns = ['target', 'predict']

In [71]:
len(df[df['target'] == df['predict']])

50

- 50개 정도밖에 못 맞힘

- 혹시 relu의 문제인가?

In [72]:
y_result = []

In [73]:
for i in range(iris['data'].shape[0]):
    
    # 입력층 노드
    start = iris['data'][i]
    
    # 입력층 -> 은닉층
    u1 = np.dot(start, W1) + b1
    f1 = leaky(u1)
    
    # 은닉층 -> 은닉층
    u2 = np.dot(f1, W2) + b2
    f2 = leaky(u2)
    
    # 은닉층 -> 출력층
    u3 = np.dot(f2, W3) + b3
    f3 = leaky(u3)
    
    # 출력층 -> 항등함수
    result = softmax(f3)
    result = result.argmax()    
    
    y_result.append(result)

In [74]:
target = pd.DataFrame(target)
result = pd.DataFrame(y_result)

In [75]:
df = pd.concat([target, result], axis=1)

In [76]:
df.columns = ['target', 'predict']

In [77]:
df

,target,predict
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
145,2,1
146,2,1
147,2,1
148,2,1


In [78]:
len(df[df['target'] == df['predict']])

82

- 살짝 나음

### 알고있는 활성화함수 다 써보기

In [79]:
relu_result = []
leaky_result = []
sigmoid_result = []
tanh_result = []
last = [relu_result, leaky_result, sigmoid_result, tanh_result]
activation_function = [ReLU, leaky, sigmoid, tanh]

In [80]:
for num, func in enumerate(activation_function):

    activ = func
    
    for i in range(iris['data'].shape[0]):
        
        # 입력층 노드
        start = iris['data'][i]

        # 입력층 -> 은닉층
        u1 = np.dot(start, W1) + b1
        f1 = activ(u1)

        # 은닉층 -> 은닉층
        u2 = np.dot(f1, W2) + b2
        f2 = activ(u2)

        # 은닉층 -> 출력층
        u3 = np.dot(f2, W3) + b3
        f3 = activ(u3)

        # 출력층 -> 항등함수
        result = softmax(f3)
        result = result.argmax()    
        
        last[num].append(result)

In [81]:
target = iris['target']

In [82]:
target = pd.DataFrame(target)
relu_result = pd.DataFrame(relu_result)
leaky_result = pd.DataFrame(leaky_result)
sigmoid_result = pd.DataFrame(sigmoid_result)
tanh_result = pd.DataFrame(tanh_result)

In [83]:
df = pd.concat([target, relu_result, leaky_result, sigmoid_result, tanh_result], axis=1)

In [84]:
df

,0,0,0,0,0
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
145,2,0,1,1,0
146,2,0,1,1,0
147,2,0,1,1,0
148,2,0,1,1,0


In [85]:
df.columns = ['target', 'relu_result', 'leaky_result', 'sigmoid_result', 'tanh_result']

In [86]:
len(df[df['target'] == df['relu_result']])

50

In [87]:
len(df[df['target'] == df['leaky_result']])

82

In [88]:
len(df[df['target'] == df['sigmoid_result']])

100

In [89]:
len(df[df['target'] == df['tanh_result']])

49

- 어떤 활성화함수를 써야하는지도 중요하지만 그보다 중요한건 적합한 가중치와 편향을 찾아야함

- 결론: 성능을 올려야하는 실습이 아닌 활성화함수를 구현할 줄 알고 신경망을 직접 쌓을 줄 아는지를 공부해보는 실습이기에 여기서 끝내겠습니다